<font size="6"><stron>REPORT</strong></font>
<h1>Finding the best location to open an Indian Restaurant in Boston</h1>
<ol>
    <li>
        <h2>Background</h2>
        <p>
            USA has always been favorite among Indians when it comes to overseas education and the number of students from India only keeps increasing every year. According to the information in the public domain like the one found on wikipedia, the US government estimates over more than 2,00,000 F visa Indian students in the United States. 
Massachusetts has consistently been positioned in the top 10 US states for international students. The dynamic city of Boston offers cultural diversity, and stimuli of big city but on a moderately-paced scale. Massachusetts boasts of the oldest and arguably most prestigious college in the country, Harvard University, which was founded in 1636. Both Harvard University and the neighboring university of Massachusetts Institute of Technology consistently rank among the world’s best universities year after year. Additionally, there are over 120 quality institutions of higher learning in Massachusetts, with over 50 in the Boston metropolitan area alone! And Indians make the most count among the total international students in and around Boston. 
Typically, Indian food is all about the varied spices and condiments which makes this cuisine more flavourful and healthy. This project is on opening up Indian restaurant in a neighborhood in Boston city which can cater lot of people and is profitable for the entrepreneur.
        </p>
    </li>
    <li>
        <h2>Business problem</h2>
        <p>
            The objective of this capstone project is to find a best location for the entrepreneur to open an authentic Indian Restaurant in Boston, MA, USA. For this project, I will apply various data science methods and tools along with machine learning algorithms such as clustering, aiming to provide solutions to answer the business question: What would be the best location in Boston for an entrepreneur to open an Indian restaurant?
        </p>
    </li>
    <li>
        <h2>Target Audience</h2>
        <p>
            This project mainly targets someone who wants to open an Indian Restaurant in Boston.
        </p>
    </li>
    <li>
        <h2>Data</h2>
        <p>
            To complete this project we need following data:
            <ol>
                <li>Neighborhood data of Boston MA</li>
                <li>Latitude and Longitude of the neighborhoods in Boston</li>
                <li>Venue data related to restaurants in Boston city</li>
            </ol>
            This will help us find neighborhoods that are more suitable to open an Indian Restaurant.
        </p>
    </li>
    <li>
        <h2>Data extraction</h2>
        <p>
            <ol>
                <li>
                    Neighborhood data of Boston MA : Scraping wikipedia page to obtain Boston neighborhood data
                </li>
                <li>Latitude and Longitude of the neighborhoods in Boston : Using Geodcoder package to obtain latitude and longitude data of these neighborhoods
                </li>
                <li>Venue data related to restaurants in Boston city : Using Foursquare API to get venue data related to these neighborhoods
                </li>
            </ol> 
        </p>
    </li>
    <li>
        <h2>Methodology</h2>
        <h3>Download libraries needed for this project</h3>
    </li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import urllib.request

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: ...working... done

## Package Plan ##

  environment location: E:\MSDA\Software Tools\Anaconda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.4                |   py38h32f6830_2         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.9 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0




==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: E:\MSDA\Software Tools\Anaconda

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::bra

<h3>Scrape the Wikipedia page and obtain the data that is in the list and to transform the data into a list</h3>

In [44]:
#To get the list of states in America
url = "https://en.wikipedia.org/wiki/Neighborhoods_in_Boston#List_of_places_and_squares_within_neighborhood_areas"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
content = soup.find('div', {"class": "div-col columns column-width"})
neighborhood_Boston = []
for i in content.findAll('li'):
    neighborhood_Boston.append(i.text)

Check the data in the list.

In [45]:
neighborhood_Boston

['Allston',
 'Back Bay',
 'Bay Village',
 'Beacon Hill',
 'Brighton',
 'Charlestown',
 'Chinatown/Leather District',
 'Dorchester (divided for planning purposes into Mid Dorchester and Dorchester)',
 'Downtown',
 'East Boston',
 'Fenway Kenmore (includes Longwood)',
 'Hyde Park',
 'Jamaica Plain',
 'Mattapan',
 'Mission Hill',
 'North End',
 'Roslindale',
 'Roxbury',
 'South Boston',
 'South End',
 'West End',
 'West Roxbury']

<h3>Data Cleaning</h3>
<h4>
    <ul>
        <li>Remove the description in parenthesis</li>
        <li>Extract 'Chinatown' from Chinatown/Leather District and 'Fenway' from 'Fenway Kenmore' so that the latitude and longitude can be found</li>
    </ul>
</h4>

In [46]:
neighborhood = []
for n in neighborhood_Boston:
    new = n.split('(')
    neighborhood.append(new[0])
neighborhood[6] = neighborhood[6].split('/')[0]
neighborhood[10] = neighborhood[10].split(' ')[0]

To check the list and see if the data is as expected

In [47]:
neighborhood

['Allston',
 'Back Bay',
 'Bay Village',
 'Beacon Hill',
 'Brighton',
 'Charlestown',
 'Chinatown',
 'Dorchester ',
 'Downtown',
 'East Boston',
 'Fenway',
 'Hyde Park',
 'Jamaica Plain',
 'Mattapan',
 'Mission Hill',
 'North End',
 'Roslindale',
 'Roxbury',
 'South Boston',
 'South End',
 'West End',
 'West Roxbury']

<h3>
Use geopy library to get the latitude and longitude values of Boston.
</h3>

In order to define an instance of the geocoder, we need to define a user_agent and name our agent boston_explorer, as shown below.

In [48]:
#Get the latitude and longitude values for each neighborhood
data = []
for s in neighborhood:
    geolocator = Nominatim(user_agent="boston_explorer")
    location = geolocator.geocode(s+', Boston, Massachusetts')
    latitude = location.latitude
    longitude = location.longitude
    data.append([s,latitude,longitude])

Create a dataframe named 'Boston' with the 'Neighborhood', 'Latitude', and 'Longitude' values

In [49]:
Boston = pd.DataFrame(data, columns = ['Neighborhood', 'Latitude','Longitude'])

To check the dataframe

In [50]:
Boston

,Neighborhood,Latitude,Longitude
0,Allston,42.355434,-71.132127
1,Back Bay,42.350707,-71.079730
2,Bay Village,42.350011,-71.066948
3,Beacon Hill,42.358708,-71.067829
4,Brighton,42.350097,-71.156442
5,Charlestown,42.377875,-71.061996
6,Chinatown,42.349405,-71.063582
7,Dorchester,42.297320,-71.074495
8,Downtown,42.358602,-71.063875
9,East Boston,42.375097,-71.039217


<h3>Lets visualize the Boston neighborhoods on the map</h3>

In [51]:
geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode('Boston, Massachusetts')
blatitude = location.latitude
blongitude = location.longitude
print(f'{blatitude} {blongitude}')

42.3602534 -71.0582912


In [52]:
# create map of Boston
map_boston = folium.Map(location=[blatitude, blongitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Boston['Latitude'], Boston['Longitude'], Boston['Neighborhood']):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

Next, we utilize the Foursquare API to explore the neighborhoods and segment them.

<h3>Define Foursquare Credentials and Version</h3>

In [53]:
CLIENT_ID = '0FENVP43DTETNE4SXYJ43A4MOYXNPEA0FWJMCW35IJIUVLDO' #  Foursquare ID
CLIENT_SECRET = 'THNPS1TLJOITTFKT4IAU0YV3F2PBREOE1L3UUEJOYTPU2GXM' #  Foursquare Secret
VERSION = '20200723'

<h3>Explore Neighborhoods in Boston</h3>
Let's create a function to find the top 50 (max for Foursquare API) venues in Boston neighborhoods

In [94]:
def getNeighborhoodList(neighborhood, latitudes, longitudes):
    LIMIT = 100 #max 100 results
    radius = 10000 #the area to cover
    venue_list=[]
    for neigh, lat, lng in zip(neighborhood, latitudes, longitudes):
        cat = '4d4b7105d754a06374d81259'
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/search?categoryId={cat}&query=Restaurant&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}'
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venue_list.append([(
            neigh,
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    venue_list = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
    venue_list.columns = [
                  'Neighborhood',           
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(venue_list)

Call the getNeighborhoodList funtion for all the neighborhoods in Boston and store the data returned in a dataframe named boston_places

In [95]:
boston_places = getNeighborhoodList(      
                                    neighborhood = Boston['Neighborhood'],
                                    latitudes=Boston['Latitude'],
                                    longitudes=Boston['Longitude']
                                    )

Check the size of the resulting dataframe

In [96]:
print(boston_places.shape)
boston_places.head()

(1100, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Allston,42.355434,-71.132127,Crystal Restaurant,42.355736,-71.130946,Russian Restaurant
1,Allston,42.355434,-71.132127,Le's Vietnamese Restaurant,42.353022,-71.131408,Vietnamese Restaurant
2,Allston,42.355434,-71.132127,B.D. Chilipa Restaurant,42.352249,-71.125527,Szechuan Restaurant
3,Allston,42.355434,-71.132127,Ariana Restaurant,42.362593,-71.138953,Afghan Restaurant
4,Allston,42.355434,-71.132127,The Restaurant At The End Of The Universe,42.359273,-71.129803,Snack Place


<h3>Data Correction</h3>

Lets check the venue categories

In [97]:
boston_places["Venue Category"].unique()

array(['Russian Restaurant', 'Vietnamese Restaurant',
       'Szechuan Restaurant', 'Afghan Restaurant', 'Snack Place',
       'Chinese Restaurant', 'Tapas Restaurant', 'Food',
       'Burmese Restaurant', 'Korean Restaurant', 'Japanese Restaurant',
       'Middle Eastern Restaurant', 'Asian Restaurant',
       'Latin American Restaurant', 'Caribbean Restaurant', 'Restaurant',
       'Deli / Bodega', 'American Restaurant', 'Brazilian Restaurant',
       'Bar', 'Turkish Restaurant', 'Diner', 'Seafood Restaurant',
       'Portuguese Restaurant', 'Gastropub', 'Mexican Restaurant',
       'African Restaurant', 'Sushi Restaurant', 'Thai Restaurant',
       'Hotpot Restaurant', 'Bakery', 'Pizza Place', 'Indian Restaurant',
       'New American Restaurant', 'Italian Restaurant',
       'Greek Restaurant', 'South American Restaurant', 'BBQ Joint',
       'Breakfast Spot', 'French Restaurant', 'Coffee Shop',
       'Mediterranean Restaurant', 'Peruvian Restaurant',
       'Malay Restaurant', 'I

In [98]:
print(f'No. of unique categories: {len(boston_places["Venue Category"].unique())}.')

No. of unique categories: 50.


In the venue categories we have additional erroneuos categories i.e., 'Food', 'Restaurant' and 'Diner'.
<ul>
    <li>Drop the data entries with 'Venue Category' as  'Food', 'Restaurant' and 'Diner'.</li>
    <li>Reason: The generic categories fail to give a better insight in the cuisine culture</li>
</ul>

In [99]:
for cat in ['Food','Restaurant','Diner']:
    indexNames = boston_places[ boston_places['Venue Category'] == cat].index
    boston_places.drop(indexNames , inplace=True)

Check the size boston_places so that we know the number of rows (data enteries) after data correction.

In [100]:
print(boston_places.shape)
boston_places.head()

(831, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Allston,42.355434,-71.132127,Crystal Restaurant,42.355736,-71.130946,Russian Restaurant
1,Allston,42.355434,-71.132127,Le's Vietnamese Restaurant,42.353022,-71.131408,Vietnamese Restaurant
2,Allston,42.355434,-71.132127,B.D. Chilipa Restaurant,42.352249,-71.125527,Szechuan Restaurant
3,Allston,42.355434,-71.132127,Ariana Restaurant,42.362593,-71.138953,Afghan Restaurant
4,Allston,42.355434,-71.132127,The Restaurant At The End Of The Universe,42.359273,-71.129803,Snack Place


Recheck the venue category

In [101]:
print(f'We have {len(boston_places["Venue Category"].unique())} unique categories after data correction.')

We have 47 unique categories after data correction.


Let's check how many restaurants were returned for each neighborhood

In [102]:
boston_places[['Neighborhood','Venue Name']].groupby('Neighborhood').count()

,Venue Name
Neighborhood,
Allston,39
Back Bay,36
Bay Village,38
Beacon Hill,39
Brighton,39
Charlestown,44
Chinatown,40
Dorchester,27
Downtown,41


In [103]:
print(f'So, we have {Boston.shape[0]} neighborhoods in total and we received data for {boston_places[["Neighborhood","Venue Name"]].groupby("Neighborhood").count().shape[0]} neighborhoods.')

So, we have 22 neighborhoods in total and we received data for 22 neighborhoods.


We have data for all the neighborhoods and nothing is missing.

<h3>Analyze Each Neighborhood for the cuisines</h3>

In [104]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_places[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhood'] = boston_places['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Belgian Restaurant,Brazilian Restaurant,...,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Allston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Allston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Allston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Allston,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Allston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [105]:
boston_mean = boston_onehot.groupby('Neighborhood').mean().reset_index()
#us_groupby
boston_mean

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bar,Belgian Restaurant,Brazilian Restaurant,...,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sports Bar,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Allston,0.025641,0.025641,0.051282,0.025641,0.000000,0.000000,0.025641,0.00000,0.051282,...,0.000000,0.000000,0.000000,0.000000,0.025641,0.025641,0.076923,0.025641,0.025641,0.025641
1,Back Bay,0.000000,0.027778,0.111111,0.000000,0.000000,0.027778,0.027778,0.00000,0.027778,...,0.027778,0.000000,0.000000,0.000000,0.027778,0.000000,0.027778,0.055556,0.000000,0.000000
2,Bay Village,0.000000,0.000000,0.105263,0.078947,0.000000,0.026316,0.026316,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.026316,0.052632,0.026316,0.026316
3,Beacon Hill,0.000000,0.000000,0.076923,0.025641,0.025641,0.025641,0.025641,0.00000,0.025641,...,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.025641,0.025641,0.000000,0.025641
4,Brighton,0.025641,0.025641,0.051282,0.025641,0.000000,0.000000,0.025641,0.00000,0.051282,...,0.000000,0.000000,0.000000,0.000000,0.025641,0.025641,0.076923,0.051282,0.025641,0.025641
5,Charlestown,0.000000,0.000000,0.090909,0.000000,0.022727,0.022727,0.000000,0.00000,0.045455,...,0.000000,0.000000,0.000000,0.022727,0.045455,0.000000,0.045455,0.022727,0.000000,0.000000
6,Chinatown,0.000000,0.000000,0.125000,0.100000,0.000000,0.025000,0.025000,0.00000,0.000000,...,0.000000,0.025000,0.000000,0.000000,0.025000,0.000000,0.025000,0.025000,0.025000,0.025000
7,Dorchester,0.000000,0.037037,0.111111,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.074074,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.111111
8,Downtown,0.000000,0.000000,0.097561,0.024390,0.024390,0.024390,0.024390,0.00000,0.024390,...,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.024390,0.024390,0.000000,0.024390
9,East Boston,0.000000,0.000000,0.119048,0.000000,0.000000,0.023810,0.000000,0.00000,0.000000,...,0.023810,0.000000,0.023810,0.023810,0.023810,0.000000,0.023810,0.000000,0.000000,0.023810


Examine the new dataframe size.

In [106]:
boston_mean.shape

(22, 48)

Let's check the top 5 most common venues for each neighborhood.

In [107]:
num_top_venues = 5

for s in boston_mean['Neighborhood']:
    print("----"+s+"----")
    temp = boston_mean[boston_mean['Neighborhood'] == s].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                       venue  freq
0         Chinese Restaurant  0.21
1           Tapas Restaurant  0.08
2          Korean Restaurant  0.05
3        American Restaurant  0.05
4  Latin American Restaurant  0.05


----Back Bay----
                 venue  freq
0  American Restaurant  0.11
1   Chinese Restaurant  0.11
2   Italian Restaurant  0.08
3      Thai Restaurant  0.06
4     Greek Restaurant  0.06


----Bay Village----
                 venue  freq
0   Chinese Restaurant  0.24
1   Seafood Restaurant  0.13
2  American Restaurant  0.11
3     Asian Restaurant  0.08
4   Italian Restaurant  0.08


----Beacon Hill----
                 venue  freq
0   Italian Restaurant  0.13
1   Chinese Restaurant  0.10
2  American Restaurant  0.08
3   Seafood Restaurant  0.08
4          Pizza Place  0.05


----Brighton----
                  venue  freq
0    Chinese Restaurant  0.13
1      Tapas Restaurant  0.08
2  Caribbean Restaurant  0.08
3   American Restaurant  0.05
4       Thai Restaur

<h3>Let's put that into a pandas dataframe</h3>

Below is a function to sort the venues in descending order.

In [108]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [109]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = boston_mean['Neighborhood']

for ind in np.arange(boston_mean.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_mean.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Chinese Restaurant,Tapas Restaurant,Latin American Restaurant,American Restaurant,Korean Restaurant,Brazilian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Gastropub,Deli / Bodega
1,Back Bay,American Restaurant,Chinese Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Seafood Restaurant,Caribbean Restaurant,Korean Restaurant,Latin American Restaurant,Hotpot Restaurant
2,Bay Village,Chinese Restaurant,Seafood Restaurant,American Restaurant,Italian Restaurant,Asian Restaurant,Thai Restaurant,Vietnamese Restaurant,Gastropub,Hotpot Restaurant,Turkish Restaurant
3,Beacon Hill,Italian Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,Brazilian Restaurant,Gastropub,Hotpot Restaurant
4,Brighton,Chinese Restaurant,Tapas Restaurant,Caribbean Restaurant,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,American Restaurant,Brazilian Restaurant,Gastropub,Turkish Restaurant
5,Charlestown,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,American Restaurant,Seafood Restaurant,Latin American Restaurant,Tapas Restaurant,Brazilian Restaurant,Sushi Restaurant,Pizza Place
6,Chinatown,Chinese Restaurant,Seafood Restaurant,American Restaurant,Asian Restaurant,Italian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Latin American Restaurant,Breakfast Spot,Gastropub
7,Dorchester,Caribbean Restaurant,Latin American Restaurant,Vietnamese Restaurant,American Restaurant,Chinese Restaurant,Spanish Restaurant,African Restaurant,Tapas Restaurant,Italian Restaurant,Seafood Restaurant
8,Downtown,Italian Restaurant,American Restaurant,Chinese Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,French Restaurant,Hotpot Restaurant,Deli / Bodega
9,East Boston,Mexican Restaurant,Chinese Restaurant,American Restaurant,Italian Restaurant,Latin American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Bakery,Deli / Bodega,Hotpot Restaurant


<h3>K-Means Clustering</h3>

Run k-means to cluster the neighborhood into 5 clusters.

In [110]:
# set number of clusters
kclusters = 5

boston_clustering = boston_mean.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 2, 0, 2, 1, 4, 2, 3])

In [111]:
# add clustering labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhood_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Allston,Chinese Restaurant,Tapas Restaurant,Latin American Restaurant,American Restaurant,Korean Restaurant,Brazilian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Gastropub,Deli / Bodega
1,2,Back Bay,American Restaurant,Chinese Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Seafood Restaurant,Caribbean Restaurant,Korean Restaurant,Latin American Restaurant,Hotpot Restaurant
2,1,Bay Village,Chinese Restaurant,Seafood Restaurant,American Restaurant,Italian Restaurant,Asian Restaurant,Thai Restaurant,Vietnamese Restaurant,Gastropub,Hotpot Restaurant,Turkish Restaurant
3,2,Beacon Hill,Italian Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,Brazilian Restaurant,Gastropub,Hotpot Restaurant
4,0,Brighton,Chinese Restaurant,Tapas Restaurant,Caribbean Restaurant,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,American Restaurant,Brazilian Restaurant,Gastropub,Turkish Restaurant


In [112]:
boston_merged = Boston

# merge the data to add latitude/longitude for each state and capital
boston_merged = boston_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

boston_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,42.355434,-71.132127,0,Chinese Restaurant,Tapas Restaurant,Latin American Restaurant,American Restaurant,Korean Restaurant,Brazilian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Gastropub,Deli / Bodega
1,Back Bay,42.350707,-71.079730,2,American Restaurant,Chinese Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Seafood Restaurant,Caribbean Restaurant,Korean Restaurant,Latin American Restaurant,Hotpot Restaurant
2,Bay Village,42.350011,-71.066948,1,Chinese Restaurant,Seafood Restaurant,American Restaurant,Italian Restaurant,Asian Restaurant,Thai Restaurant,Vietnamese Restaurant,Gastropub,Hotpot Restaurant,Turkish Restaurant
3,Beacon Hill,42.358708,-71.067829,2,Italian Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,Brazilian Restaurant,Gastropub,Hotpot Restaurant
4,Brighton,42.350097,-71.156442,0,Chinese Restaurant,Tapas Restaurant,Caribbean Restaurant,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,American Restaurant,Brazilian Restaurant,Gastropub,Turkish Restaurant
5,Charlestown,42.377875,-71.061996,2,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,American Restaurant,Seafood Restaurant,Latin American Restaurant,Tapas Restaurant,Brazilian Restaurant,Sushi Restaurant,Pizza Place
6,Chinatown,42.349405,-71.063582,1,Chinese Restaurant,Seafood Restaurant,American Restaurant,Asian Restaurant,Italian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Latin American Restaurant,Breakfast Spot,Gastropub
7,Dorchester,42.297320,-71.074495,4,Caribbean Restaurant,Latin American Restaurant,Vietnamese Restaurant,American Restaurant,Chinese Restaurant,Spanish Restaurant,African Restaurant,Tapas Restaurant,Italian Restaurant,Seafood Restaurant
8,Downtown,42.358602,-71.063875,2,Italian Restaurant,American Restaurant,Chinese Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,French Restaurant,Hotpot Restaurant,Deli / Bodega
9,East Boston,42.375097,-71.039217,3,Mexican Restaurant,Chinese Restaurant,American Restaurant,Italian Restaurant,Latin American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Bakery,Deli / Bodega,Hotpot Restaurant


<h3>Analyzing the Clusters</h3>

<h3>Cluster 0</h3>

In [113]:
cluster0 = boston_merged.loc[boston_merged['Cluster Labels'] == 0, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]
cluster0

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,42.355434,Tapas Restaurant,Latin American Restaurant,American Restaurant,Korean Restaurant,Brazilian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Gastropub,Deli / Bodega
4,42.350097,Tapas Restaurant,Caribbean Restaurant,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,American Restaurant,Brazilian Restaurant,Gastropub,Turkish Restaurant
10,42.345365,Tapas Restaurant,Caribbean Restaurant,Italian Restaurant,Indian Restaurant,Latin American Restaurant,Bar,Brazilian Restaurant,Bakery,Mexican Restaurant
12,42.309820,Caribbean Restaurant,Latin American Restaurant,American Restaurant,Tapas Restaurant,Deli / Bodega,Turkish Restaurant,Italian Restaurant,Indian Restaurant,Gastropub
14,42.332560,Caribbean Restaurant,Latin American Restaurant,Tapas Restaurant,Mexican Restaurant,Turkish Restaurant,Indian Restaurant,Hotpot Restaurant,Gastropub,Deli / Bodega
16,42.291209,Caribbean Restaurant,Tapas Restaurant,American Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Breakfast Spot,Gastropub,Deli / Bodega,Russian Restaurant
17,42.324843,Chinese Restaurant,Latin American Restaurant,Tapas Restaurant,Mexican Restaurant,Spanish Restaurant,Russian Restaurant,Turkish Restaurant,Middle Eastern Restaurant,Deli / Bodega
18,42.333431,American Restaurant,Italian Restaurant,Caribbean Restaurant,Seafood Restaurant,African Restaurant,Latin American Restaurant,Peruvian Restaurant,Pizza Place,Deli / Bodega
19,42.341310,American Restaurant,Thai Restaurant,Italian Restaurant,Latin American Restaurant,Seafood Restaurant,Caribbean Restaurant,Brazilian Restaurant,Middle Eastern Restaurant,Deli / Bodega
21,42.279265,Latin American Restaurant,Chinese Restaurant,Tapas Restaurant,Italian Restaurant,American Restaurant,Breakfast Spot,Middle Eastern Restaurant,Indian Restaurant,Fast Food Restaurant


In [114]:
cluster0.describe(include='all')

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,10.000000,10,10,10,10,10,10,10,10,10
unique,NaN,5,6,5,8,9,9,7,6,6
top,NaN,Tapas Restaurant,Latin American Restaurant,Tapas Restaurant,Mexican Restaurant,Latin American Restaurant,Breakfast Spot,Brazilian Restaurant,Gastropub,Deli / Bodega
freq,NaN,3,4,3,2,2,2,3,3,5
mean,42.326333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.025411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,42.279265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,42.313576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,42.332995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,42.344352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Cluster 0 conclusion:</h4>
Cuisine that dominates: Tapas Restaurants

<h3>Cluster 1</h3>

In [124]:
cluster1 = boston_merged.loc[boston_merged['Cluster Labels'] == 1, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]
cluster1

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,42.350011,Seafood Restaurant,American Restaurant,Italian Restaurant,Asian Restaurant,Thai Restaurant,Vietnamese Restaurant,Gastropub,Hotpot Restaurant,Turkish Restaurant
6,42.349405,Seafood Restaurant,American Restaurant,Asian Restaurant,Italian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Latin American Restaurant,Breakfast Spot,Gastropub


In [125]:
cluster1.describe(include='all')

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,2.000000,2,2,2,2,2,2,2,2,2
unique,NaN,1,1,2,2,2,2,2,2,2
top,NaN,Seafood Restaurant,American Restaurant,Asian Restaurant,Asian Restaurant,Vietnamese Restaurant,Turkish Restaurant,Gastropub,Hotpot Restaurant,Gastropub
freq,NaN,2,2,1,1,1,1,1,1,1
mean,42.349708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.000428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,42.349405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,42.349557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,42.349708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,42.349860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Cluster 1 conclusion:</h4>
Cuisine that dominates: Seafood Restaurants

<h3>Cluster 2</h3>

In [126]:
cluster2 = boston_merged.loc[boston_merged['Cluster Labels'] == 2, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]
cluster2

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,42.350707,Chinese Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Seafood Restaurant,Caribbean Restaurant,Korean Restaurant,Latin American Restaurant,Hotpot Restaurant
3,42.358708,Chinese Restaurant,American Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,Brazilian Restaurant,Gastropub,Hotpot Restaurant
5,42.377875,Italian Restaurant,Mexican Restaurant,American Restaurant,Seafood Restaurant,Latin American Restaurant,Tapas Restaurant,Brazilian Restaurant,Sushi Restaurant,Pizza Place
8,42.358602,American Restaurant,Chinese Restaurant,Seafood Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,French Restaurant,Hotpot Restaurant,Deli / Bodega
15,42.365097,Chinese Restaurant,Seafood Restaurant,American Restaurant,Latin American Restaurant,Pizza Place,Peruvian Restaurant,Brazilian Restaurant,Gastropub,Greek Restaurant
20,42.363919,American Restaurant,Chinese Restaurant,Greek Restaurant,Seafood Restaurant,Portuguese Restaurant,Pizza Place,Peruvian Restaurant,Middle Eastern Restaurant,Gastropub


In [127]:
cluster2.describe(include='all')

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,6.000000,6,6,6,6,6,6,6,6,6
unique,NaN,3,5,3,4,4,5,4,5,5
top,NaN,Chinese Restaurant,Chinese Restaurant,American Restaurant,Greek Restaurant,Pizza Place,Vietnamese Restaurant,Brazilian Restaurant,Gastropub,Hotpot Restaurant
freq,NaN,3,2,2,2,3,2,3,2,2
mean,42.362485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.009097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,42.350707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,42.358629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,42.361314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,42.364803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Cluster 2 conclusion:</h4>
Cuisine that dominates: Chinese Restaurants

<h3>Cluster 3</h3>

In [128]:
cluster3 = boston_merged.loc[boston_merged['Cluster Labels'] == 3, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]
cluster3

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,42.375097,Chinese Restaurant,American Restaurant,Italian Restaurant,Latin American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Bakery,Deli / Bodega,Hotpot Restaurant


In [129]:
cluster3.describe(include='all')

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,1.000000,1,1,1,1,1,1,1,1,1
unique,NaN,1,1,1,1,1,1,1,1,1
top,NaN,Chinese Restaurant,American Restaurant,Italian Restaurant,Latin American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Bakery,Deli / Bodega,Hotpot Restaurant
freq,NaN,1,1,1,1,1,1,1,1,1
mean,42.375097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,42.375097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,42.375097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,42.375097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,42.375097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Cluster 3 conclusion:</h4>
Cuisine that dominates: Chinese Restaurant followed by American Restaurant

<h3>Cluster 4</h3>

In [130]:
cluster4 = boston_merged.loc[boston_merged['Cluster Labels'] == 4, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]
cluster4

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,42.297320,Latin American Restaurant,Vietnamese Restaurant,American Restaurant,Chinese Restaurant,Spanish Restaurant,African Restaurant,Tapas Restaurant,Italian Restaurant,Seafood Restaurant
11,42.255654,Chinese Restaurant,American Restaurant,Latin American Restaurant,Italian Restaurant,Tapas Restaurant,Szechuan Restaurant,Asian Restaurant,Southern / Soul Food Restaurant,Fast Food Restaurant
13,42.267566,Chinese Restaurant,American Restaurant,Latin American Restaurant,African Restaurant,Tapas Restaurant,Sushi Restaurant,Italian Restaurant,Southern / Soul Food Restaurant,Vietnamese Restaurant


In [131]:
cluster4.describe(include='all')

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,3.000000,3,3,3,3,3,3,3,3,3
unique,NaN,2,2,2,3,2,3,3,2,3
top,NaN,Chinese Restaurant,American Restaurant,Latin American Restaurant,Chinese Restaurant,Tapas Restaurant,Sushi Restaurant,Asian Restaurant,Southern / Soul Food Restaurant,Fast Food Restaurant
freq,NaN,2,2,2,1,2,1,1,2,1
mean,42.273513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.021460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,42.255654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,42.261610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,42.267566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,42.282443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4>Cluster 4 conclusion:</h4>
Cuisine that dominates: Chinese Restaurant and Latin American Restaurant

<h3>Visualizing the Clusters</h3>

In [133]:
# create map
bostonmap_clusters = folium.Map(location=[blatitude, blongitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['Latitude'], boston_merged['Longitude'], boston_merged['Neighborhood'], boston_merged['Cluster Labels']):
    label_clus = ''
    if cluster==0:
        label_clus = 'Tapas Restaurant'
    elif cluster==1:
        label_clus = 'Seafood Restaurant'
    elif cluster==2:
        label_clus = 'Chinese Restaurants'
    elif cluster==3:
        label_clus = 'Chinese + American Restaurants'
    elif cluster==4:
        label_clus = 'Chinese + American Restaurants'
        
    label = folium.Popup(str(poi) + ', '+label_clus, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(bostonmap_clusters)
       
bostonmap_clusters

<h4>Map Legend
<ul>
    <li>Cluster 0: Red    : Tapas Restaurants</li>
    <li>Cluster 1: Purple : Seafood Restaurants</li>
    <li>Cluster 2: Blue   : Chinese Restaurants</li>
    <li>Cluster 3: Green  : Chinese & American Restaurants</li>
    <li>Cluster 4: Yellow : Chinese & American Restaurants</li>
</ul>
<h4>

<h3>7. Observation</h3>
<p>
From the above clusters, it is evident that there are no Indian Restaurants in the top 10 most common venue.
We cannot infer whether there are Indian Restaurants in Boston but are not popular or there are only few Indian Restaurants in Boston neighborhoods. 
One thing is clear that if a person wants to successfully set up an Indian Restuarant business in Boston, he/she/they can open up in any of the neighborhoods provided if they cater quality/tasty food to people. 
Additionally, if we look at the clusters, we can see that there are many asian restaurants that are pretty popular in many of the neighborhoods. Therefore, the person can open an Indian restaurants which can also appease asian population by providing chinese, thai and japanese food options.
It will also be success if the Indian Restaurant has bar option which provide for the general population by offering happy hour drinks which can bring profit to business. 
</p>

So based on observations and analysis, my recommendations are as follows:
<ol>
    <li>Cluster 0 locations- Indian Restaurants</li>
    <li>Cluster 1 locations- Indian Restaurants</li>
    <li>Cluster 2 locations- Indian Restaurants/Indo-chinese Restaurants</li>
    <li>Cluster 3 locations- Indian Restaurants/Indo-chinese Restaurants</li>
    <li>Cluster 4 locations- Indian Restaurants/Indo-chinese Restaurants</li>
</ol>
<h3>8. Conclusion</h3>
<p>
In this project, we looked at neighborhoods that has a variety of restaurants and discovered locations to open an Indian Restaurant with extra options to make business profitable. We scrapped wikepedia page that has information about Boston neighborhoods. We cleansed data and by using geospatial libraries. Using Foursquare API, we collected the top 50 restaurants and their types for each location within a radius 500 meters from the given latitude-longitude. Then we grouped collected restaurants by location and by taking the mean of the frequency of occurrence of each type, prepared them for clustering. Finally we clustered restaurants by the k-means algorithm and analyze the top 10 most common restaurants in each cluster, making insightful observations. Eventually, we visualized clusters on the map, thus showing the best locations for opening the chosen type of restaurant. This type of analysis can be applied to any city of your choice that has available geospatial information. This type of analysis can be applied to any type of venue (shopping, clubs, etc.) that is available in Foursquare database.
</p>